In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import cv2
from keras.models import Sequential
from keras.layers import Dense, Conv2D,MaxPooling2D, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


2023-10-24 16:31:42.466675: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#Image data generator 

training_datagen= ImageDataGenerator(rescale=1.0/255)
validation_datagen=ImageDataGenerator(rescale=1.0/255)

In [4]:
#Fetch and preprocess all the training images

training_data = training_datagen.flow_from_directory(
    directory='Dataset/train',
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",  
    class_mode="categorical"
)

Found 28709 images belonging to 7 classes.


In [5]:
# Fetch and preprocess all the validation Images

testing_data=validation_datagen.flow_from_directory(
    directory='Dataset/test',
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 7178 images belonging to 7 classes.


#Creating the Model


In [6]:

model=Sequential() 
  
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)) )
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))
    
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(7,activation='softmax'))



In [7]:
cv2.ocl.setUseOpenCL(False)

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 22, 22, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                   

In [15]:
model.fit_generator(
    training_data,
    epochs=50,
    steps_per_epoch=28709//64,
    validation_data=testing_data,
    validation_steps=7178//64
)

    
    

Epoch 1/50


/var/folders/y5/0pk6n8ss36qc7jpnwlwc_cd40000gp/T/ipykernel_663/427085033.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


448/448 [==============================] - 125s 276ms/step - loss: 1.7080 - accuracy: 0.3137 - val_loss: 1.5254 - val_accuracy: 0.4206
Epoch 2/50
448/448 [==============================] - 121s 269ms/step - loss: 1.4360 - accuracy: 0.4454 - val_loss: 1.3022 - val_accuracy: 0.4940
Epoch 3/50
448/448 [==============================] - 119s 266ms/step - loss: 1.2812 - accuracy: 0.5088 - val_loss: 1.2193 - val_accuracy: 0.5324
Epoch 4/50
448/448 [==============================] - 118s 263ms/step - loss: 1.1982 - accuracy: 0.5449 - val_loss: 1.1891 - val_accuracy: 0.5428
Epoch 5/50
448/448 [==============================] - 119s 265ms/step - loss: 1.1384 - accuracy: 0.5655 - val_loss: 1.1437 - val_accuracy: 0.5640
Epoch 6/50
448/448 [==============================] - 126s 280ms/step - loss: 1.0827 - accuracy: 0.5915 - val_loss: 1.1117 - val_accuracy: 0.5727
Epoch 7/50
448/448 [==============================] - 130s 290ms/step - loss: 1.0293 - accuracy: 0.6108 - val_loss: 1.0891 - val_accura

In [16]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [18]:
model.save_weights('model_weights.h5')

In [20]:
model.save('model.keras')